# NLP - Norvig spell corrector

Norvig spell corrector is an elementary yet powerful spell corrector that finds out all the words one or two or multiple edit distances away from the given word. It finally assigns correct spelling to the word which has the maximum probability of occurence in a given corpus.

In [4]:
import re
from collections import Counter

In [5]:
# function to tokenise words
def words(document):
    "Convert text to lower case and tokenise the document"
    return re.findall(r'\w+', document.lower())

In [6]:
# create a frequency table of all the words of the document
all_words = Counter(words(open('big.txt').read()))

In [7]:
# check frequency of a random word, say, 'chair'
all_words['chair']

135

In [8]:
# look at top 10 frequent words
all_words.most_common(10)

[('the', 79809),
 ('of', 40024),
 ('and', 38312),
 ('to', 28765),
 ('in', 22023),
 ('a', 21124),
 ('that', 12512),
 ('he', 12401),
 ('was', 11410),
 ('it', 10681)]

In [9]:
def edits_one(word):
    "Create all edits that are one edit away from `word`."
    alphabets    = 'abcdefghijklmnopqrstuvwxyz'
    splits     = [(word[:i], word[i:])                   for i in range(len(word) + 1)]
    deletes    = [left + right[1:]                       for left, right in splits if right]
    inserts    = [left + c + right                       for left, right in splits for c in alphabets]
    replaces   = [left + c + right[1:]                   for left, right in splits if right for c in alphabets]
    transposes = [left + right[1] + right[0] + right[2:] for left, right in splits if len(right)>1]
    return set(deletes + inserts + replaces + transposes)

In [10]:
def edits_two(word):
    "Create all edits that are two edits away from `word`."
    return (e2 for e1 in edits_one(word) for e2 in edits_one(e1))

In [11]:
def known(words):
    "The subset of `words` that appear in the `all_words`."
    return set(word for word in words if word in all_words)

In [12]:
def possible_corrections(word):
    "Generate possible spelling corrections for word."
    return (known([word]) or known(edits_one(word)) or known(edits_two(word)) or [word])

In [13]:
def prob(word, N=sum(all_words.values())): 
    "Probability of `word`: Number of appearances of 'word' / total number of tokens"
    return all_words[word] / N

In [14]:
print(len(set(edits_one("monney"))))
print(edits_one("monney"))

336
{'monneyq', 'monneoy', 'mvnney', 'mhonney', 'monnrey', 'omnney', 'monnuey', 'monley', 'mwnney', 'myonney', 'monyey', 'jmonney', 'moniney', 'monnei', 'monnen', 'mronney', 'monneny', 'monxney', 'mnoney', 'mdonney', 'mznney', 'monxey', 'monwey', 'monnecy', 'monned', 'mtonney', 'mozney', 'modney', 'mounney', 'moenney', 'monneya', 'monnez', 'aonney', 'monvey', 'monnmy', 'mouney', 'monnedy', 'monnea', 'mojney', 'mwonney', 'monyney', 'monneyy', 'monneyu', 'fonney', 'eonney', 'mponney', 'monmey', 'mlnney', 'monrney', 'monneyb', 'monnuy', 'monneyc', 'monneym', 'umonney', 'cmonney', 'mhnney', 'monnney', 'monnety', 'monoey', 'kmonney', 'monuney', 'wonney', 'konney', 'mmonney', 'monnpey', 'monnewy', 'vonney', 'mosney', 'moncey', 'emonney', 'monnzey', 'moiney', 'monnej', 'mononey', 'meonney', 'zmonney', 'tmonney', 'monneyn', 'uonney', 'moneey', 'momney', 'mpnney', 'mmnney', 'dmonney', 'monlney', 'montey', 'momnney', 'monntey', 'montney', 'mjnney', 'gmonney', 'monneqy', 'yonney', 'motney', 'movn

In [15]:
print(known(edits_one("monney")))

{'money', 'monkey'}


In [16]:
# Let's look at words that are two edits away
print(len(set(edits_two("monney"))))
print(known(edits_one("monney")))

51013
{'money', 'monkey'}


In [17]:
# Let's look at possible corrections of a word
print(possible_corrections("monney"))

{'money', 'monkey'}


In [18]:
# Let's look at probability of a word
print(prob("money"))
print(prob("monkey"))

0.0002922233626303688
5.378344097491451e-06


In [19]:
def spell_check(word):
    "Print the most probable spelling correction for `word` out of all the `possible_corrections`"
    correct_word = max(possible_corrections(word), key=prob)
    if correct_word != word:
        return "Did you mean " + correct_word + "?"
    else:
        return "Correct spelling."

In [20]:
# test spell check
print(spell_check("monney"))

Did you mean money?
